# 02: Create Agent with LangGraph

Create a multi-tool calling agent using Databricks LangGraph pattern.

**Pattern**: Based on [Databricks LangGraph Agent Guide](https://docs.databricks.com/aws/en/notebooks/source/generative-ai/responses-agent-langgraph.html)

**Prerequisites**:
- UC Functions registered (run `01-register-uc-functions.ipynb` first)

## Install Dependencies

In [0]:
%pip install --quiet --upgrade mlflow databricks-langchain langgraph langchain-core
dbutils.library.restartPython()

## Setup and Configuration

In [0]:
import sys
sys.path.append('../src')

import mlflow
from databricks_langchain import UCFunctionToolkit, ChatDatabricks
from langgraph.prebuilt import create_react_agent
from config.agent_config import AGENT_CONFIG, get_all_uc_function_names
from config.prompts import SYSTEM_PROMPT

print("✅ Imports successful")

## Create UC Function Toolkit

Load the registered UC Functions as tools.

In [0]:
# Get UC Function names from config
function_names = get_all_uc_function_names()

print(f"Loading UC Functions: {function_names}\n")

# Create toolkit
toolkit = UCFunctionToolkit(function_names=function_names)

print(f"✅ Toolkit created")
print(f"Available tools: {[tool.name for tool in toolkit.tools]}")

## Initialize LLM

In [0]:
# Create ChatDatabricks LLM
llm = ChatDatabricks(
    endpoint=AGENT_CONFIG["model_endpoint"],
    # temperature=AGENT_CONFIG["temperature"],
    # max_tokens=AGENT_CONFIG["max_tokens"]
)

print(f"✅ LLM initialized: {AGENT_CONFIG['model_endpoint']}")

## Create Agent with LangGraph

Using `create_react_agent` (Databricks recommended pattern).

In [0]:
# Create agent using LangGraph
agent = create_react_agent(
    model=llm,
    tools=toolkit.tools,
    prompt=SYSTEM_PROMPT
)

print("✅ Agent created successfully")
print(f"Agent type: {type(agent)}")

## Test Agent

Quick smoke test to verify the agent works.

In [0]:
# Test query
# test_query = "What data domains do you have access to?"
test_query = "What products are at risk for overstock?"

print(f"Query: {test_query}\n")

# Invoke agent (LangGraph message format)
result = agent.invoke(
    {"messages": [("user", test_query)]}
)

# Extract response
final_message = result["messages"][-1]
response_text = final_message.content

print(f"Response:\n{response_text}")
print(f"\n✅ Agent test PASSED")

## Log Agent to MLflow

Register the agent for deployment and tracking.

In [0]:
# Set MLflow experiment
username = spark.sql("SELECT current_user()").collect()[0][0]
experiment_name = f"/Users/{username}/ml/experiments/multi-tool-calling-agent"

mlflow.set_experiment(experiment_name)
print(f"MLflow experiment: {experiment_name}\n")

# Log agent
with mlflow.start_run(run_name="langgraph-agent") as run:
    # Log parameters
    mlflow.log_params(AGENT_CONFIG)
    mlflow.log_param("agent_type", "langgraph_react")
    mlflow.log_param("tools", function_names)
    
    # Log model
    mlflow.langchain.log_model(
        lc_model=agent,
        name="agent",
        registered_model_name="multi_tool_calling_agent"
    )
    
    run_id = run.info.run_id
    print(f"✅ Agent logged to MLflow")
    print(f"Run ID: {run_id}")
    print(f"Model URI: runs:/{run_id}/agent")

## Next Step

✅ Agent created and logged to MLflow!

**Next**: Open `03-test-agent.ipynb` to run comprehensive tests.